<a href="https://colab.research.google.com/github/zjimf/Cerebral-Stroke-Prediction/blob/master/Cerebral_Stroke_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Loading Dataset

In [ ]:
!gdown --id '1ysAKkeJoSCYQ-b152N1VY3XOnOFBJXB0' --output Cerebral_Stroke_Prediction.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ysAKkeJoSCYQ-b152N1VY3XOnOFBJXB0
To: /content/Cerebral_Stroke_Prediction.csv
100% 2.64M/2.64M [00:00<00:00, 141MB/s]


##Import

In [ ]:
import pandas as pd

In [ ]:
file_path = './Cerebral_Stroke_Prediction.csv'
df = pd.read_csv(file_path)

In [ ]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
